# Regression Model

## Load Data

In [1]:
%autosave 20

#basic library
import pandas as pd
import numpy as np
import collections
from collections import defaultdict


#model training
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# visulization
import matplotlib.pyplot as plt


# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Autosaving every 20 seconds


In [2]:
df = pd.read_excel('databaseForFunction.xlsx', index_col=0)

In [3]:
df.head()

,State,County,Year,Month,PDensity,Population,SNAP_Applications,numberOfWorkers,numberOfDisaster,google_calfresh,google_food_bank,google_food_pantry,google_food_stamps,google_supplemental,google_snap,date_time,last_date_time,last_snap,last_worker,last_disaster,last_google_calfresh,last_google_food_bank,last_google_food_pantry,last_google_food_stamps,last_google_supplemental,last_google_snap
0,California,Alameda,2019,1,1898.5,1559308,5515,45,0,24.00,28.75,38.00,55.50,54.50,0.0,2019-01-01,2018-12-01,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0
1,California,Alameda,2019,2,1898.5,1559308,4478,397,0,27.25,27.75,38.75,31.25,36.75,0.0,2019-02-01,2019-01-01,5515,45,0,24.00,28.75,38.00,55.50,54.50,0.0
2,California,Alameda,2019,3,1898.5,1559308,5041,191,0,19.60,23.20,33.80,31.80,35.20,0.0,2019-03-01,2019-02-01,4478,397,0,27.25,27.75,38.75,31.25,36.75,0.0
3,California,Alameda,2019,4,1898.5,1559308,5253,808,0,23.25,22.50,35.50,22.00,29.25,0.0,2019-04-01,2019-03-01,5041,191,0,19.60,23.20,33.80,31.80,35.20,0.0
4,California,Alameda,2019,5,1898.5,1559308,8074,502,0,35.00,26.75,31.50,29.50,31.25,0.0,2019-05-01,2019-04-01,5253,808,0,23.25,22.50,35.50,22.00,29.25,0.0


In [4]:
# split CA and texas using dummy

# create dummy variables
dummies = pd.get_dummies(df['State'], prefix='State')

# concatenate the dummy variables with the original dataframe
df = pd.concat([df, dummies], axis=1)


In [5]:
# drop State_Texas since using dummy

df = df.drop(columns='State_Texas')

## Choose columns to train 
- use last month date to predict this month's snap applications

In [6]:
col_keepinmodel = list(df.columns)

In [7]:
coltomove = ['State','County','Year','Month', 'date_time','last_date_time', \
             'numberOfWorkers', 'numberOfDisaster','google_calfresh',\
             'google_food_bank','google_food_pantry','google_food_stamps',\
             'google_supplemental','google_snap',]

for c in coltomove:
    col_keepinmodel.remove(c)
    
col_keepinmodel

['PDensity',
 'Population',
 'SNAP_Applications',
 'last_snap',
 'last_worker',
 'last_disaster',
 'last_google_calfresh',
 'last_google_food_bank',
 'last_google_food_pantry',
 'last_google_food_stamps',
 'last_google_supplemental',
 'last_google_snap',
 'State_California']

In [8]:
df = df[col_keepinmodel]

In [9]:
df.head()

,PDensity,Population,SNAP_Applications,last_snap,last_worker,last_disaster,last_google_calfresh,last_google_food_bank,last_google_food_pantry,last_google_food_stamps,last_google_supplemental,last_google_snap,State_California
0,1898.5,1559308,5515,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,1
1,1898.5,1559308,4478,5515,45,0,24.00,28.75,38.00,55.50,54.50,0.0,1
2,1898.5,1559308,5041,4478,397,0,27.25,27.75,38.75,31.25,36.75,0.0,1
3,1898.5,1559308,5253,5041,191,0,19.60,23.20,33.80,31.80,35.20,0.0,1
4,1898.5,1559308,8074,5253,808,0,23.25,22.50,35.50,22.00,29.25,0.0,1


# Build the Model

#### 1. check the correlation

In [10]:
df.corr()['SNAP_Applications']

PDensity                    0.631942
Population                  0.933607
SNAP_Applications           1.000000
last_snap                   0.947601
last_worker                 0.392523
last_disaster               0.036997
last_google_calfresh        0.192687
last_google_food_bank       0.004543
last_google_food_pantry     0.011594
last_google_food_stamps     0.045971
last_google_supplemental    0.043198
last_google_snap           -0.185326
State_California            0.217959
Name: SNAP_Applications, dtype: float64

In [11]:
X = df.drop(columns=['SNAP_Applications'])
y = df.SNAP_Applications

In [12]:
features_ = df.drop(columns=['SNAP_Applications']).columns.to_list()

In [13]:
features_.insert(0,'const')

In [14]:
features_

['const',
 'PDensity',
 'Population',
 'last_snap',
 'last_worker',
 'last_disaster',
 'last_google_calfresh',
 'last_google_food_bank',
 'last_google_food_pantry',
 'last_google_food_stamps',
 'last_google_supplemental',
 'last_google_snap',
 'State_California']

#### 2. Result of the linear regression model

In [15]:
import statsmodels.api as sm
import pandas as pd

# Add constant term to X
X = sm.add_constant(X)

# Fit linear regression model
model = sm.OLS(y, X).fit()

# Print summary of regression results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:      SNAP_Applications   R-squared:                       0.925
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                 1.126e+04
Date:                Thu, 13 Apr 2023   Prob (F-statistic):               0.00
Time:                        15:32:33   Log-Likelihood:                -94157.
No. Observations:               10908   AIC:                         1.883e+05
Df Residuals:                   10895   BIC:                         1.884e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

#### 2. Scale the data and train the model

In [17]:
# # scale the data 
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# # Create a LinearRegression object and fit the training data
# reg = LinearRegression().fit(X_train, y_train)

# # Print the coefficients and intercept
# print('Coefficients:', reg.coef_)
# print('Intercept:', reg.intercept_)

# # Make predictions on the test set
# y_pred_test = reg.predict(X_test)

# # Make predictions on the train set
# y_pred_train = reg.predict(X_train)

Coefficients: [   0.         -145.75210443 2320.94388073 2820.72536309 -108.06517367
  145.78767689  -50.00789742   87.1501502   -64.76475801  139.15353215
 -206.26635881  -63.23347177 -161.95605882]
Intercept: 1313.4639001319706


In [18]:
# coef = reg.coef_

# coef_dic = collections.defaultdict(float)

# for i in range(len(coef)):
#     coef_dic[features_[i]] = coef[i]

In [19]:
# pd.DataFrame(coef_dic, index=['coef_']).T

,coef_
const,0.000000
PDensity,-145.752104
Population,2320.943881
last_snap,2820.725363
last_worker,-108.065174
last_disaster,145.787677
last_google_calfresh,-50.007897
last_google_food_bank,87.150150
last_google_food_pantry,-64.764758
last_google_food_stamps,139.153532


#### 3. evalute the result

In [20]:
# # evalute the train data

# from sklearn.metrics import mean_squared_error
# mse = mean_squared_error(y_train, y_pred_train)

# from sklearn.metrics import r2_score
# r2 = r2_score(y_train, y_pred_train)

# from sklearn.metrics import mean_absolute_error
# mae = mean_absolute_error(y_train, y_pred_train)

# from sklearn.metrics import mean_squared_error
# rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))

# train_loss = {'MSE':[mse], 'R2':[r2], 'MAE':[mae], 'RMSE':[rmse]}

In [21]:
# # evalute the test data

# from sklearn.metrics import mean_squared_error
# mse = mean_squared_error(y_test, y_pred_test)

# from sklearn.metrics import r2_score
# r2 = r2_score(y_test, y_pred_test)

# from sklearn.metrics import mean_absolute_error
# mae = mean_absolute_error(y_test, y_pred_test)

# from sklearn.metrics import mean_squared_error
# rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

# test_loss = {'MSE':[mse], 'R2':[r2], 'MAE':[mae], 'RMSE':[rmse]}

In [23]:
# # visulization

# import matplotlib.pyplot as plt

# # convert the data to pandas dataframes
# df1 = pd.DataFrame(train_loss)
# df2 = pd.DataFrame(test_loss)
                   
# index_labels=['train_loss','test_loss']

# # merge the dataframes
# df = pd.concat([df1,df2], ignore_index=True)
# df = df.rename(index = {0:'train_loss', 1:'test_loss'})

# df